In [1]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-cpu
!pip install Chromadb
!pip install google-search-results
!pip install wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.4 MB ? eta -:--:--
     ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.4 MB 2.6 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.4/1.4 MB 21.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.0/399.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... do

In [21]:
# 印象笔记有记录
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["SERPAPI_API_KEY"] = ""
os.environ["SERPER_API_KEY"] = ""


#1.Agent 调用 tools

##1.1Agent 一般调用

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("北京奥运会的召开年份除以4，得到的结果求0.5 次方是多少?")



> Entering new AgentExecutor chain...
 I need to calculate the result of the equation
Action: Calculator
Action Input: Beijing Olympic Games year divided by 4, then 0.5 power
Observation: Answer: 22.40535650240808
Thought: I now know the final answer
Final Answer: 22.40535650240808

> Finished chain.


'22.40535650240808'

##1.2在对话中调用

In [ ]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=0)
agent = initialize_agent(tools, chat_model, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("北京奥运会的召开年份除以4，得到的结果求0.5 次方是多少?")



> Entering new AgentExecutor chain...
Thought: This is a math question that involves division and exponentiation. I can use the calculator tool to solve it.

Action:
```
{
  "action": "Calculator",
  "action_input": "((2008 / 4) ^ 0.5)"
}
```
Observation: Answer: 22.40535650240808
Thought:The result of the calculation is 22.40535650240808.
Final Answer: 22.40535650240808

> Finished chain.


'22.40535650240808'

##1.3在文档中调用
"Search"：此工具的功能是在整个文档存储中搜索与提供的查询相关的信息。这就像你在Wikipedia中输入一个关键词进行全局搜索，返回的结果可能是一个或多个与关键词相关的文章或段落。

"Lookup"：此工具的功能是查找文档存储中的特定信息。这通常意味着你已经知道你想要查找的具体文档或信息，并使用此工具直接获取该文档。这就像你在Wikipedia中直接访问一个特定的页面。

In [ ]:
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="当搜索时使用",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="当查找时使用",
    ),
]

llm = OpenAI(temperature=0, model_name="text-davinci-002")
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

In [ ]:
question = "中华人们共和国什么时候成立的？"

react.run(question)



> Entering new AgentExecutor chain...

Thought: I need to search for the date when the People's Republic of China was founded.

Action: Search[People's Republic of China]

Observation: China (Chinese: 中国; pinyin: Zhōngguó), officially the People's Republic of China (PRC), is a country in East Asia. It is the world's second-most populous country with a population exceeding 1.4 billion. China spans the equivalent of five time zones and borders fourteen countries by land, tied with Russia as having the most of any country in the world. With an area of nearly 9.6 million square kilometres (3,700,000 sq mi), it is the world's third largest country by total land area. The country consists of 22 provinces, five autonomous regions, four municipalities, and two special administrative regions (Hong Kong and Macau). The national capital is Beijing, and the most populous city and largest financial center is Shanghai.
The region that is now China has been inhabited since the Paleolithic era, with

OutputParserException: ignored

#2.Conversational
使用专门为对话优化的代理。其他代理通常优化为使用工具来找出最佳回应
使用一种特殊类型的代理（conversational-react-description），该代理预计会配合一个记忆组件来使用。

In [ ]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain import GoogleSerperAPIWrapper
from langchain.agents import initialize_agent

search = GoogleSerperAPIWrapper()

tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="搜索当前的事件"
    ),
]

memory = ConversationBufferMemory(memory_key="chat_history")

llm=OpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

agent_chain.run(input="小明今年24岁")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: 小明今年24岁，真是时间飞逝啊！

> Finished chain.


'小明今年24岁，真是时间飞逝啊！'

In [ ]:
agent_chain.run(input="北京的温度加上小明的年纪是多少")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: 北京的温度是80°F，小明今年24岁，所以总共是104°F。

> Finished chain.


'北京的温度是80°F，小明今年24岁，所以总共是104°F。'

#3.OpenAI Multi Functions Agent
OpenAI的多功能代理（OpenAI Multi Functions Agent）来处理用户的提问

In [ ]:
from langchain import GoogleSerperAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

search = GoogleSerperAPIWrapper()

# Define a list of tools offered by the agent
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful when you need to answer questions about current events. You should ask targeted questions.",
    ),
]

mrkl = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_MULTI_FUNCTIONS, verbose=True
)

mrkl.run("北京和上海的天气如何?")



> Entering new AgentExecutor chain...

Invoking: `Search` with `{'tool_input': '北京天气'}`


80°F
Invoking: `Search` with `{'tool_input': '上海天气'}`


83°F北京的天气为80°F，上海的天气为83°F。

> Finished chain.


'北京的天气为80°F，上海的天气为83°F。'

#4.Self ask with search
用自问自答的方式拆解问题

In [ ]:
from langchain import OpenAI, GoogleSerperAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

llm = OpenAI(temperature=0)
search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)
#Java 语言的创始者是谁？ 【X】
#【X】的家乡在哪里？
self_ask_with_search.run(
    "Java 语言的创建者的家乡是哪里?"
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: Who is the creator of Java language?
Intermediate answer: James Gosling
Follow up: Where is James Gosling from?
Intermediate answer: Calgary, Canada
So the final answer is: Calgary, Canada

> Finished chain.


'Calgary, Canada'

#5.Multi-Input Tools
自定义一个乘法工具，然后在agent中调用。通过这个例子，大家可以定义自己的工具，可以是读取文件，也可以调用其他的API或者接口。


In [ ]:
from langchain import OpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool

llm = OpenAI(temperature=0)

#定义工具调用的函数
def multiplier(a: float, b: float) -> float:
    """Multiply the provided floats."""
    return a * b
#加法运算
def plus(a: float, b: float) -> float:
    """plus the provided floats."""
    return a + b

#定义工具
tool = StructuredTool.from_function(multiplier)
tool2 = StructuredTool.from_function(plus)

# 初始化Agent 设置tool
agent_executor = initialize_agent(
    [tool,tool2],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
agent_executor.run("3+5 等于多少")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "plus",
  "action_input": {
    "a": 3,
    "b": 5
  }
}
```

Observation: 8.0
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "3+5=8"
}
```

> Finished chain.


'3+5=8'

#6.Tool Input Schema

"Tool Input Schema" 是用于定义并校验工具输入的一个模式或规范。

安装了一个Python包tldextract，然后使用它来验证URL的域名是否在许可的域名列表中。它使用ToolInputSchema类来定义并验证工具输入的格式。这个类有一个url字段，并有一个根验证器（@root_validator）用于检查URL的域名是否在_APPROVED_DOMAINS列表中。如果不在，它将抛出一个错误。

In [ ]:
pip install tldextract > /dev/null

In [ ]:
from typing import Any, Dict

from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.tools.requests.tool import RequestsGetTool, TextRequestsWrapper
from pydantic import BaseModel, Field, root_validator


import tldextract

#创建一个域名访问的白名单
_APPROVED_DOMAINS = {
    "google",
    "wikipedia",
}

#验证输入的内容是否在白名单里面
class ToolInputSchema(BaseModel):
    url: str = Field(...)

    @root_validator
    def validate_query(cls, values: Dict[str, Any]) -> Dict:
        url = values["url"]
        domain = tldextract.extract(url).domain
        if domain not in _APPROVED_DOMAINS:
            raise ValueError(
                f"域名 {domain} 不在白名单中:"
                f" {sorted(_APPROVED_DOMAINS)}"
            )
        print("通过白名单校验")
        return values

tool = RequestsGetTool(
    args_schema=ToolInputSchema, requests_wrapper=TextRequestsWrapper()
)

agent = initialize_agent(
    [tool], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)

In [ ]:
answer = agent.run(" www.google.com 的主题是什么")
print(answer)

通过白名单校验


InvalidRequestError: ignored

#7.Human-in-the-loop Tool Validation
人为参与验证

使用ShellTool：首先，该文本介绍了ShellTool的使用方法。ShellTool在这里被用来执行echo Hello World!命令，并输出Hello World!。

添加人工审批：然后，文本介绍了如何为工具添加人工审批的方法。通过为工具添加默认的HumanApprovalCallbackHandler，可以确保用户必须手动批准输入到工具的每个输入，然后才会实际执行命令。在这里，它被用来在执行ls /usr命令之前进行人工审批。


In [ ]:
from langchain.callbacks import HumanApprovalCallbackHandler
from langchain.tools import ShellTool

tool = ShellTool()

print(tool.run("echo Hello World!"))


Hello World!



/usr/local/lib/python3.10/dist-packages/langchain/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


In [ ]:
tool = ShellTool(callbacks=[HumanApprovalCallbackHandler()])

print(tool.run("ls /usr"))


Do you approve of the following input? Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no.

ls /usr
Y
bin
colab
games
grte
include
lib
lib32
lib64
libexec
libx32
local
sbin
share
src



In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

def _should_check(serialized_obj: dict) -> bool:
    #判断输入的命令是否是shell
    return serialized_obj.get("name") == "terminal"

def _approve(_input: str) -> bool:
    if _input == "echo 'Hello World'":
        return True
    msg = (
        "你是否批准以下输入？"
        "除了 'Y'/'Yes'（不区分大小写）之外的任何内容都将视为不批准。"
    )
    msg += "\n\n" + _input + "\n"
    resp = input(msg)
    return resp.lower() in ("yes", "y")


callbacks = [HumanApprovalCallbackHandler(should_check=_should_check, approve=_approve)]

In [ ]:
llm = OpenAI(temperature=0)
tools = load_tools(["wikipedia", "llm-math", "terminal"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

agent.run(
    "现在是2023年。中华人民共和国成立多少年了？",
    callbacks=callbacks,
)

"The People's Republic of China was founded on October 1, 1949."

In [ ]:
agent.run("print 'Hello World' in the terminal", callbacks=callbacks)

'Hello World'

In [ ]:
agent.run("list all directories in /private", callbacks=callbacks)

你是否批准以下输入？除了 'Y'/'Yes'（不区分大小写）之外的任何内容都将视为不批准。

ls /private
Y
你是否批准以下输入？除了 'Y'/'Yes'（不区分大小写）之外的任何内容都将视为不批准。

ls /
Y


'bin, boot, content, cuda-keyring_1.0-1_all.deb, datalab, dev, etc, home, lib, lib32, lib64, libx32, media, mnt, NGC-DL-CONTAINER-LICENSE, opt, proc, python-apt, root, run, sbin, srv, sys, tmp, tools, usr, var'

#8.Running Agent as an Iterator
这段代码表示将问题的处理过程分成多个步骤，每次循环处理一个步骤。

agent.iter(question)这段代码是将问题的处理过程进行迭代。agent.iter()是一个生成器函数，它不会一次性完成所有的处理过程，而是生成一个迭代器，每次迭代返回问题处理的下一个步骤。

在每个循环中，step表示当前的处理步骤，可以包含中间结果或者其他的信息。每次循环都可能涉及到一些操作，例如调用工具函数、检查结果等。这样可以让你有机会在每个步骤中插入自定义的处理逻辑，比如进行检查或者干预处理过程。

In [ ]:
import pydantic
from langchain.agents import AgentExecutor, initialize_agent, AgentType
from langchain.schema import AgentFinish
from langchain.agents.tools import Tool
from langchain import LLMMathChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

#定义质数的数组，序号：质数
primes = {998: 3, 999: 8, 1000: 11}

# 定义一个数据模型，用于描述计算器输入。这个模型有一个字段：question，类型是字符串
class CalculatorInput(pydantic.BaseModel):
    question: str = pydantic.Field()

# 定义一个数据模型，用于描述获取质数的输入。这个模型有一个字段：n，类型是整数
class PrimeInput(pydantic.BaseModel):
    n: int = pydantic.Field()

# 定义一个函数，用于判断一个整数是否是质数
#质数的定义是指一个大于1的自然数，除了1和它本身之外，没有其他正因数能够整除它。
def is_prime(n: int) -> bool:
    if n <= 1 or (n % 2 == 0 and n > 2):
        return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0:
            return False
    return True

# 定义一个函数，用于从字典中获取指定的质数，并返回这个质数的字符串形式
def get_prime(n: int, primes: dict = primes) -> str:
    return str(primes.get(int(n)))

# 定义一个异步函数，用于从字典中获取指定的质数，并返回这个质数的字符串形式
async def aget_prime(n: int, primes: dict = primes) -> str:
    return str(primes.get(int(n)))

# 定义一个工具列表，包含两个工具：获取质数的工具和计算器工具
tools = [
    # 获取质数的工具
    Tool(
        name="GetPrime",  # 工具名称
        func=get_prime,  # 工具执行的函数
        description="返回第n个质数的工具",  # 工具描述
        args_schema=PrimeInput,  # 工具输入的数据模型
        coroutine=aget_prime,  # 工具执行的异步函数
    ),
    # 计算器工具
    Tool.from_function(
        name="Calculator",  # 工具名称
        func=llm_math_chain.run,  # 工具执行的函数
        description="数学计算工具",  # 工具描述
        args_schema=CalculatorInput,  # 工具输入的数据模型
        coroutine=llm_math_chain.arun,  # 工具执行的异步函数
    ),
]

# 初始化一个智能代理，这个代理包含上述定义的两个工具
agent = initialize_agent(
    tools,  # 代理的工具列表
    llm,  # 代理的语言模型
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # 代理类型
    verbose=True  # 是否打印详细的日志
)


In [ ]:
question = "请判断序号 998，999，1000存放的数字是否是质数?"

# agent 利用迭代，每次处理一个序号的数字
for step in agent.iter(question):

    # 获取中间步骤的输出，如果有输出，将其赋值给'output'，并进入if语句块。
    # ':=' 这种写法是Python 3.8引入的赋值表达式（也被称为海象操作符）。
    if output := step.get("intermediate_step"):

        # 将'output'（它是一个列表）中的第一个元素（一个元组）解包成'action'和'value'。
        # 'action'代表当前的操作，'value'是操作的结果。
        action, value = output[0]

        # 检查当前的操作是否使用了名为'GetPrime'的工具。
        if action.tool == "GetPrime":

            # 如果使用了'GetPrime'，那么打印一条消息，说明正在检查'value'是否为质数。
            print(f"正在检查{value} 是否是质数...")

            # 利用'is_prime'函数检查'value'是否为质数。如果不是，assert会触发一个AssertionError。
            # 没有
            if is_prime(int(value)):
              print(value +"是质数")
            else:
              print(value +"不是质数")

        # 询问用户是否希望继续执行下一步。输入的结果将被赋值给变量'_continue'。
        _continue = input("Should the agent continue (Y/n)?:\n")

        # 如果用户输入的不是'Y'（代表“是”），那么就跳出for循环，结束执行。
        if _continue != "Y":
            break




> Entering new AgentExecutor chain...
We can use the GetPrime tool to check if the numbers at positions 998, 999, and 1000 are prime.
Action: GetPrime
Action Input: 998
Observation: 3
Thought:正在检查3 是否是质数...
3是质数
Should the agent continue (Y/n)?:
Y
The number at position 998 is 3.
Action: GetPrime
Action Input: 999
Observation: 8
Thought:正在检查8 是否是质数...
8不是质数
Should the agent continue (Y/n)?:
Y
The number at position 999 is 8.
Action: GetPrime
Action Input: 1000
Observation: 11
Thought:正在检查11 是否是质数...
11是质数
Should the agent continue (Y/n)?:
Y
The number at position 1000 is 11.
Final Answer: The numbers at positions 998, 999, and 1000 are not all prime.

> Finished chain.


#9.Combine agents and vector stores

结合代理和向量存储。如果你已经将数据导入到向量存储，然后希望以代理方式与其进行交互，推荐的方法是创建一个 RetrievalQA，并在总代理中使用它作为工具。你可以使用多个不同的 vectordbs，并使用代理作为它们之间的路由器。

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import WebBaseLoader
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

llm = OpenAI(temperature=0)

In [4]:
from langchain.document_loaders import TextLoader

#向量数据的加载与嵌入
loader = TextLoader("story.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="story")

story = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

#调用web 上的文件，内容
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

tools = [
    Tool(
        name="story",
        func=story.run,
        description="",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [10]:
agent.run("ruff工具是否能够帮助莉莉营救泰格")



> Entering new AgentExecutor chain...
 我需要了解ruff工具的功能
Action: Ruff QA System
Action Input: 什么是ruff工具的功能？
Observation:  Ruff是一个代码检查工具，它可以检测代码中的错误和不规范，并可以自动修复一些错误。它可以用来替换Flake8，并且可以与Mypy，Pyright或Pyre等类型检查器一起使用，以提供更详细的反馈。
Thought: 根据ruff工具的功能，它不能帮助莉莉营救泰格
Final Answer: 不，ruff工具不能帮助莉莉营救泰格。

> Finished chain.


'不，ruff工具不能帮助莉莉营救泰格。'

#10.Async API
利用 asyncio 库为Agents提供异步支持

In [17]:
import asyncio
import time

from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.callbacks.stdout import StdOutCallbackHandler
from langchain.callbacks.tracers import LangChainTracer
from aiohttp import ClientSession

questions = [
    "2019年美国公开赛男子决赛的冠军是谁？他的年龄的0.334次方是多少？",
"奥利维亚·王尔德的男朋友是谁？他现在的年龄的0.23次方是多少？",
"最近的一场一级方程式大奖赛的冠军是谁？他们的年龄的0.23次方是多少？"
]

In [18]:
llm = OpenAI(temperature=0)
tools = load_tools(["google-serper", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

s = time.perf_counter()
#串行执行
for q in questions:
    agent.run(q)
elapsed = time.perf_counter() - s
print(f"串行执行 in {elapsed:0.2f} seconds.")



> Entering new AgentExecutor chain...
 I need to find out who won the 2019 US Open men's final and then calculate the 0.334 power of his age.
Action: google_serper
Action Input: 2019 US Open men's final winner
Observation: Rafael Nadal defeated Daniil Medvedev in the final, 7–5, 6–3, 5–7, 4–6, 6–4 to win the men's singles tennis title at the 2019 US Open. It was his fourth US ... Champion: Rafael Nadal. Draw: 128 (16 Q / 8 WC). Seeds: 32. Runner-up: Daniil Medvedev. Rafael Nadal won his 19th career Grand Slam title, and his fourth US Open crown, by surviving an all-time comback effort from Daniil ... In a repeat of the 2019 Rogers Cup, Rafael Nadal won the men's singles title, defeating first time Grand Slam finalist Daniil Medvedev in a 5-set thriller for ... Daniil Medvedev vs Rafael Nadal Full Match | 2019 US ... Duration: 4:47:32. Posted: Mar 20, 2020. How Rafael Nadal won his 19th Grand Slam title | US Open ... Duration: 10:04. Posted: Nov 27, 2019. Huge respect to Daniil Medved

In [15]:
llm = OpenAI(temperature=0)
tools = load_tools(["google-serper", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

s = time.perf_counter()
#并行执行
tasks = [agent.arun(q) for q in questions]
await asyncio.gather(*tasks)
elapsed = time.perf_counter() - s
print(f"并行执行 {elapsed:0.2f} 秒.")



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...
 I need to find out who won the 2019 US Open men's final and then calculate the 0.334 power of his age.
Action: google_serper
Action Input: 2019 US Open men's final winner I need to find out who won the most recent Formula 1 race and then calculate their age to the power of 0.23.
Action: google_serper
Action Input: "most recent Formula 1 race winner" I need to find out who Oscar Wilde's boyfriend is and then calculate the 0.23 power of his age.
Action: google_serper
Action Input: "Oscar Wilde's boyfriend"
Observation: Grand Prix, Date, Winner, Car, Laps, Time. Bahrain, 05 Mar 2023, Max Verstappen VER, Red Bull Racing Honda RBPT, 57, 1:33:56.736. Lewis Hamilton has won 103 Grands Prix during his career. He won 21 races with McLaren and has won 82 with Mercedes. Lewis Hamilton holds the record for the ... Michael Schumacher (top left) and Lewis Hamilton (top right) have

#11.Custom multi-action agent

创建自定义代理的指导。一个代理主要由两部分组成：工具和代理类本身。工具是代理可以使用的工具，代理类则决定采取何种行动。下面代码介绍了如何创建一个能预测/执行多步骤的自定义代理。

In [30]:
from langchain.agents import Tool, AgentExecutor, BaseMultiActionAgent
from langchain import OpenAI, GoogleSerperAPIWrapper

def random_word(query: str) -> str:
    print("\n输出一段文字!")
    return "我是随机文字生成器"

search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="RandomWord",
        func=random_word,
        description="call this to get a random word.",
    ),
]

from typing import List, Tuple, Any, Union
from langchain.schema import AgentAction, AgentFinish

#自定义的agent
class FakeAgent(BaseMultiActionAgent):
    """Fake Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """
        Args:
            intermediate_steps: 已经进行的代理步骤和观察结果列表
            **kwargs: 用户的输入

        Returns:
            返回一个代理行动列表或者结束代理
        """
         # 如果代理刚开始执行（即还没有进行任何步骤），那么选择执行'Search'和'RandomWord'这两个工具
        if len(intermediate_steps) == 0:
            return [
                AgentAction(tool="Search", tool_input=kwargs["input"], log=""),
                AgentAction(tool="RandomWord", tool_input=kwargs["input"], log=""),
            ]
        # 如果已经执行了至少一个步骤，那么结束代理的执行
        else:
            return AgentFinish(return_values={"output": "agent结束"}, log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """
        Args:
            intermediate_steps: 已经进行的代理步骤和观察结果列表
            **kwargs: 用户的输入

        Returns:
            返回一个代理行动列表或者结束代理
        """
        # 如果代理刚开始执行（即还没有进行任何步骤），那么选择执行'Search'和'RandomWord'这两个工具
        if len(intermediate_steps) == 0:
            return [
                AgentAction(tool="Search", tool_input=kwargs["input"], log=""),
                AgentAction(tool="RandomWord", tool_input=kwargs["input"], log=""),
            ]
        # 如果已经执行了至少一个步骤，那么结束代理的执行
        else:
            return AgentFinish(return_values={"output": "agent结束"}, log="")

agent = FakeAgent()

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [31]:
agent_executor.run("2023年世界人口有多少?")



> Entering new AgentExecutor chain...
中国日报网6月22日电（孙若男）根据联合国每年人口调查报告的最新数据及预测，2023年世界人口将突破80亿大关。
输出一段文字!
我是随机文字生成器

> Finished chain.


'agent结束'